In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2 as cv
# Изменять размер картинок до загрузки
from skimage.transform import resize
import skimage.io as io

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Flatten, SimpleRNN, LSTM, GRU, Conv2D, MaxPool2D
from keras.models import Sequential

from keras import backend
from keras import metrics
from keras import preprocessing
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
data_dir = os.path.join(os.pardir, os.pardir,'data')

x = []       
y = []
shape = (640,360)#решейп это кто

for fullpath, dirs, files in os.walk(data_dir):
    if len(dirs) == 0:
        dirname = fullpath.split(os.path.sep)[-1]
        if dirname == 'open_img':
            for file in files:
                image = io.imread(os.path.join(fullpath,file))
                image = resize(image, shape)
                x.append(image)
        else:
            for file in files:
                image = io.imread(os.path.join(fullpath, file))
                image = resize(image, shape)
                y.append(image)

x = np.array(x)
y = np.array(y)

In [ ]:
x.shape[-3:]

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Resizing

input_shape = x.shape[-3:]
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

cropped_model = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer('block1_conv2').output)
cropped_model.trainable = False

input_part = Input(input_shape)
mm = cropped_model(input_part, training=False)

model = Model(inputs=input_part, outputs=mm)
model.summary(show_trainable=True)

In [ ]:
from sklearn.model_selection import train_test_split

seed = 42
x_train, y_train, x_val, y_val = train_test_split(x, y, train_size=0.9, random_state=seed)

In [ ]:
features_train = model.predict(x_train)
features_train = features_train.reshape(-1, x_train.shape[3])
y_train = y_train.reshape(-1)

In [ ]:
np.unique(features_train)